# PyCity Schools Analysis

* 

* 

* 

In [276]:
# import libraries
import pandas as pd
import numpy as np

school_data_csv = ("../Resources/schools_complete.csv")
student_data_csv = ("../Resources/students_complete.csv")

# store csv files as dataframes
school_data = pd.read_csv(school_data_csv)
student_data = pd.read_csv(student_data_csv)

# Create single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

# Create the pass/fail bins 
grades = [0,70,100]

# Create the names for the grades and add pass/fail columns to the df for later calculations
PassFail = ["Fail","Pass"]

# Add columns to original data with pass/fail columns for later calculations
school_data_complete["passfail_math"] = pd.cut(school_data_complete["math_score"], grades, labels=PassFail, include_lowest=True)
school_data_complete["passfail_reading"] = pd.cut(school_data_complete["reading_score"], grades, labels=PassFail, include_lowest=True)

# Par down to just District Data
#school_data_district = school_data_complete.loc[school_data_complete["type"] == "District"]

school_data_complete

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget,passfail_math,passfail_reading
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635,Pass,Fail
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635,Fail,Pass
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635,Fail,Pass
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635,Fail,Fail
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635,Pass,Pass
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130,Pass,Pass
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130,Fail,Pass
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130,Pass,Pass
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130,Pass,Pass


## District Summary

In [277]:
# Calculate the total number of unique schools
school_count = len(pd.unique(school_data_complete['school_name']))
print(school_count)

15


In [278]:
# Calculate the total number of students
student_count = len(pd.unique(school_data_complete["Student ID"])) 
print(student_count)

39170


In [279]:
# Create a summary of schools and budgets
budget_all_students = pd.DataFrame(school_data_complete, columns=['school_name', 'type','budget', 'size'])

# Modify to have unique school name values
schools_budget_unique = budget_all_students.drop_duplicates(subset=["school_name"], keep='first').reset_index()
schools_budget_unique["Per Student Budget"] = schools_budget_unique["budget"] / schools_budget_unique["size"]
schools_budget_unique["Per Student Budget"] = schools_budget_unique["Per Student Budget"].map("${:}".format)

# Calculate total budget
budget_total = schools_budget_unique["budget"].sum()

schools_budget_unique
 

,index,school_name,type,budget,size,Per Student Budget
0,0,Huang High School,District,1910635,2917,$655.0
1,2917,Figueroa High School,District,1884411,2949,$639.0
2,5866,Shelton High School,Charter,1056600,1761,$600.0
3,7627,Hernandez High School,District,3022020,4635,$652.0
4,12262,Griffin High School,Charter,917500,1468,$625.0
5,13730,Wilson High School,Charter,1319574,2283,$578.0
6,16013,Cabrera High School,Charter,1081356,1858,$582.0
7,17871,Bailey High School,District,3124928,4976,$628.0
8,22847,Holden High School,Charter,248087,427,$581.0
9,23274,Pena High School,Charter,585858,962,$609.0


In [280]:
# Calculate the average (mean) math score
average_math_score = school_data_complete["math_score"].mean()
print(average_math_score)

78.98537145774827


In [281]:
# Calculate the average (mean) reading score
average_reading_score = school_data_complete["reading_score"].mean()
print(average_reading_score)

81.87784018381414


In [282]:
# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70)
passing_math_count = school_data_complete[(school_data_complete["passfail_math"] == "Pass")].count()["Student ID"]
passing_math_percentage = passing_math_count / float(student_count) * 100
print(passing_math_percentage)

72.39213683941792


In [283]:
# Calculate the percentage of students who passeed reading 
passing_reading_count = school_data_complete[(school_data_complete["passfail_reading"] == "Pass")].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_count) * 100
passing_reading_percentage

82.97166198621395

In [284]:
# Use the following to calculate the percentage of students that passed math and reading
passing_math_reading_count = school_data_complete[
    (school_data_complete["passfail_math"] == "Pass") & (school_data_complete["passfail_reading"] == "Pass")
].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100
print(overall_passing_rate)

60.801633903497574


In [285]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
d_summary = {"Total Schools":[school_count],"Total Students":[student_count],"Total Budget":[budget_total],"Average Math Score":[average_math_score],"Average Reading Score":[average_reading_score],"% Passing Math":[passing_math_percentage],"% Passing Reading":[passing_reading_percentage],"% Overall Passing":[overall_passing_rate]}
district_summary = pd.DataFrame(d_summary)

# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)
district_summary["Average Math Score"] = district_summary["Average Math Score"].map("{:,.2f}".format)
district_summary["Average Reading Score"] = district_summary["Average Reading Score"].map("{:,.2f}".format)
district_summary["% Passing Math"] = district_summary["% Passing Math"].map("{:,.2f}%".format)
district_summary["% Passing Reading"] = district_summary["% Passing Reading"].map("{:,.2f}%".format)
district_summary["% Overall Passing"] = district_summary["% Overall Passing"].map("{:,.2f}%".format)

# Display the DataFrame
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.99,81.88,72.39%,82.97%,60.80%


## School Summary

In [286]:
# Use the code provided to select the school type
school_types = school_data.set_index(["school_name"])["type"]

In [287]:
# Calculate the total student count
per_school_counts = school_data_complete.groupby(['school_name'])["Student ID"].count().reset_index()
per_school_counts = per_school_counts.rename(columns={"Student ID": "student_count"})
per_school_counts

,school_name,student_count
0,Bailey High School,4976
1,Cabrera High School,1858
2,Figueroa High School,2949
3,Ford High School,2739
4,Griffin High School,1468
5,Hernandez High School,4635
6,Holden High School,427
7,Huang High School,2917
8,Johnson High School,4761
9,Pena High School,962


In [288]:
# Calculate the total school budget and per capita spending
per_school_budget = school_data_complete.groupby(["school_name"]).mean()["budget"]
per_school_capita = per_school_budget / per_school_counts


In [289]:
# Calculate the average test scores
per_school_math = school_data_complete.groupby(['school_name'])['math_score'].mean().reset_index()
per_school_reading = school_data_complete.groupby(['school_name'])['reading_score'].mean().reset_index()
per_school_math = per_school_math.rename(columns={"math_score": "Average Math Score"})
per_school_reading = per_school_reading.rename(columns={"reading_score": "Average Reading Score"})



In [290]:
# Calculate the number of schools with math scores of 70 or higher
per_school_math["passfail_math"] = pd.cut(per_school_math["Average Math Score"], grades, labels=PassFail, include_lowest=True)
school_passing_math = len(per_school_math["passfail_math"]=="Pass")




In [291]:
# Calculate the number of schools with reading scores of 70 or higher
per_school_reading["passfail_reading"] = pd.cut(per_school_reading["Average Reading Score"], grades, labels=PassFail, include_lowest=True)
school_passing_reading = len(per_school_reading["passfail_reading"]=="Pass")

print(school_passing_reading)

15


In [292]:
# Use the provided code to calculate the schools that passed both math and reading with scores of 70 or higher
passing_math_and_reading = school_data_complete[
    (school_data_complete["passfail_reading"] == "Pass") & (school_data_complete["passfail_math"] == "Pass")]
passing_math_and_reading = passing_math_and_reading.drop(["Student ID","grade","reading_score","math_score","gender","School ID","type","size","budget"], axis=1)
passing_math_and_reading_grouped = passing_math_and_reading.groupby(["school_name"])["passfail_math"].count().reset_index()
passing_math_and_reading_grouped = passing_math_and_reading_grouped.rename(columns={"passfail_math":"passed_both"})
passing_math_and_reading_grouped

,school_name,passed_both
0,Bailey High School,2545
1,Cabrera High School,1561
2,Figueroa High School,1472
3,Ford High School,1405
4,Griffin High School,1228
5,Hernandez High School,2325
6,Holden High School,359
7,Huang High School,1456
8,Johnson High School,2371
9,Pena High School,816


In [293]:
# Calculating passing math and reading by school
passing_math = school_data_complete[school_data_complete["passfail_math"] == "Pass"]
passing_reading = school_data_complete[school_data_complete["passfail_reading"] == "Pass"]
grouped_passing_math = passing_math.groupby(['school_name'])['passfail_math'].count().reset_index()
grouped_passing_reading = passing_reading.groupby(['school_name'])['passfail_reading'].count().reset_index()
#print(grouped_passing_math)

# Create single dataset with pass/fails 
school_pass_fail_df = pd.merge(grouped_passing_math, grouped_passing_reading, how="left", on=["school_name", "school_name"])
school_pass_fail_df = pd.merge(school_pass_fail_df,passing_math_and_reading_grouped,how="left",on=["school_name","school_name"])
school_pass_fail_df = school_pass_fail_df.rename(columns={"passfail_math": "passed_math", "passfail_reading":"passed_reading"})
school_pass_fail_df

,school_name,passed_math,passed_reading,passed_both
0,Bailey High School,3216,3946,2545
1,Cabrera High School,1664,1744,1561
2,Figueroa High School,1880,2313,1472
3,Ford High School,1801,2123,1405
4,Griffin High School,1317,1371,1228
5,Hernandez High School,3001,3624,2325
6,Holden High School,387,396,359
7,Huang High School,1847,2299,1456
8,Johnson High School,3040,3727,2371
9,Pena High School,882,887,816


In [294]:
school_pass_fail_df["% Passing Math"] = school_pass_fail_df["passed_math"] / per_school_counts["student_count"] * 100
school_pass_fail_df["% Passing Reading"] = school_pass_fail_df["passed_reading"] / per_school_counts["student_count"] * 100
school_pass_fail_df["% Overall Passing"] = school_pass_fail_df["passed_both"] / per_school_counts["student_count"] * 100
school_pass_fail_df["% Passing Math"] = school_pass_fail_df["% Passing Math"].map("{:,.2f}%".format)
school_pass_fail_df["% Passing Reading"] = school_pass_fail_df["% Passing Reading"].map("{:,.2f}%".format)
school_pass_fail_df["% Overall Passing"] = school_pass_fail_df["% Overall Passing"].map("{:,.2f}%".format)
school_pass_fail_df

,school_name,passed_math,passed_reading,passed_both,% Passing Math,% Passing Reading,% Overall Passing
0,Bailey High School,3216,3946,2545,64.63%,79.30%,51.15%
1,Cabrera High School,1664,1744,1561,89.56%,93.86%,84.02%
2,Figueroa High School,1880,2313,1472,63.75%,78.43%,49.92%
3,Ford High School,1801,2123,1405,65.75%,77.51%,51.30%
4,Griffin High School,1317,1371,1228,89.71%,93.39%,83.65%
5,Hernandez High School,3001,3624,2325,64.75%,78.19%,50.16%
6,Holden High School,387,396,359,90.63%,92.74%,84.07%
7,Huang High School,1847,2299,1456,63.32%,78.81%,49.91%
8,Johnson High School,3040,3727,2371,63.85%,78.28%,49.80%
9,Pena High School,882,887,816,91.68%,92.20%,84.82%


In [295]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
per_school_summary = pd.merge(schools_budget_unique, school_pass_fail_df, how="left", on=["school_name", "school_name"])
per_school_summary = per_school_summary.set_index("school_name")
per_school_summary = per_school_summary.drop(["passed_math","passed_reading","passed_both"],axis=1)
per_school_summary["Average Math Score"] = per_school_math["Average Math Score"]
per_school_summary["Average Reading Score"] = per_school_reading["Average Reading Score"]
per_school_summary = per_school_summary.drop("index", axis=1)

# Display the DataFrame
per_school_summary

,type,budget,size,Per Student Budget,% Passing Math,% Passing Reading,% Overall Passing,Average Math Score,Average Reading Score
school_name,,,,,,,,,
Huang High School,District,1910635,2917,$655.0,63.32%,78.81%,49.91%,NaN,NaN
Figueroa High School,District,1884411,2949,$639.0,63.75%,78.43%,49.92%,NaN,NaN
Shelton High School,Charter,1056600,1761,$600.0,89.89%,92.62%,83.19%,NaN,NaN
Hernandez High School,District,3022020,4635,$652.0,64.75%,78.19%,50.16%,NaN,NaN
Griffin High School,Charter,917500,1468,$625.0,89.71%,93.39%,83.65%,NaN,NaN
Wilson High School,Charter,1319574,2283,$578.0,90.93%,93.25%,84.89%,NaN,NaN
Cabrera High School,Charter,1081356,1858,$582.0,89.56%,93.86%,84.02%,NaN,NaN
Bailey High School,District,3124928,4976,$628.0,64.63%,79.30%,51.15%,NaN,NaN
Holden High School,Charter,248087,427,$581.0,90.63%,92.74%,84.07%,NaN,NaN


In [296]:






# Count of total passes/fails for the percentage rate
total_pass_reading = (school_data_complete['passfail_reading'].value_counts()['Pass'])
#print(total_pass_reading)

total_pass_math = (school_data_complete['passfail_math'].value_counts()['Pass'])
#print(total_pass_math)

total_fail_reading = (school_data_complete['passfail_reading'].value_counts()['Fail'])
#print(total_fail_reading)

total_fail_math = (school_data_complete['passfail_math'].value_counts()['Fail'])
#print(total_fail_math)